In [94]:
import requests
import pandas as pd
from datetime import datetime
import geopandas
from shapely.geometry import Point

In [95]:
#selects previous year to calculate annual water quality averages
currentYear = datetime.now().year-1
#Water quality calculation are typically calculated for the "summer average", which is defined as June through September by the MCPA
start_date = f"{currentYear}-06-01"
end_date = f"{currentYear}-09-30"
#select data from lake sites only
site_type = "LK_STATION"
#output data as json
output_type = f"objson"
#base WISKI API URL
base_url = f"http://gis.minnehahacreek.org/KiWIS/"
#site and water quality data API urls to get site lat/longs and water quality data
site_url = f"{base_url}/KiWIS?datasource=0&format={output_type}&from={start_date}&to={end_date}&request=getwqmstationlist&service=kisters&object_type_shortname={site_type}&type=queryServices"
wq_data_url = f"{base_url}/KiWIS?datasource=0&format={output_type}&from={start_date}&to={end_date}&request=getwqmsamplevalues&service=kisters&object_type_shortname={site_type}&type=queryServices"

In [96]:
#site API get request
site_response = requests.get(site_url)
sites = site_response.json()

In [97]:
#water quality API get request
data_response = requests.get(wq_data_url)
WQ = data_response.json()

In [98]:
#create dataframes from get requests
sites_df = pd.DataFrame(sites)
WQ_df = pd.DataFrame(WQ)

#merge dataframes 
data_merge = pd.merge(WQ_df, sites_df, on="station_no", how="left")
data_merge

,station_no,station_name_x,parametertype_shortname,parametertype_name,timestamp,value,unit_symbol,station_name_y,station_latitude,station_longitude,site_no,site_name,object_type_shortname,catchment_name,sh_district
0,LAU03,Auburn Lake - East,ChlA,ChlA,2019-09-16T13:00:00.000Z,45.1,µg/l,Auburn Lake - East,44.8639,-93.67960,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;LK_STATION,---,---
1,LAU03,Auburn Lake - East,ChlA,ChlA,2019-07-22T13:15:00.000Z,48.4,µg/l,Auburn Lake - East,44.8639,-93.67960,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;LK_STATION,---,---
2,LAU03,Auburn Lake - East,ChlA,ChlA,2019-06-24T10:30:00.000Z,26.3,µg/l,Auburn Lake - East,44.8639,-93.67960,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;LK_STATION,---,---
3,LAU03,Auburn Lake - East,ChlA,ChlA,2019-08-08T08:20:00.000Z,76.8,µg/l,Auburn Lake - East,44.8639,-93.67960,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;LK_STATION,---,---
4,LAU03,Auburn Lake - East,ChlA,ChlA,2019-07-08T11:15:00.000Z,43.5,µg/l,Auburn Lake - East,44.8639,-93.67960,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;LK_STATION,---,---
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6083,LWS01,Wassermann Lake,TSS,TSS,2019-08-05T11:10:00.000Z,11.2,mg/l,Wassermann Lake,44.8410,-93.67371,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;A_STATION;LK_STATION,---,---
6084,LWS01,Wassermann Lake,TSS,TSS,2019-08-21T10:50:00.000Z,10.0,mg/l,Wassermann Lake,44.8410,-93.67371,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;A_STATION;LK_STATION,---,---
6085,LWS01,Wassermann Lake,TSS,TSS,2019-09-16T10:20:00.000Z,11.8,mg/l,Wassermann Lake,44.8410,-93.67371,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;A_STATION;LK_STATION,---,---
6086,LWS01,Wassermann Lake,TSS,TSS,2019-09-03T12:00:00.000Z,11.0,mg/l,Wassermann Lake,44.8410,-93.67371,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;A_STATION;LK_STATION,---,---


In [99]:
#data processing to obtain lake total phosphorus, clarity (SD), and algae (chl-a)
data_merge['station_no_parameter'] = data_merge['station_no'] + '-' + data_merge['parametertype_name']  
WQ_subset = data_merge[(data_merge["parametertype_name"] == "TP") | (data_merge["parametertype_name"] == "SD") | (data_merge["parametertype_name"] == "ChlA")]
WQ_subset

,station_no,station_name_x,parametertype_shortname,parametertype_name,timestamp,value,unit_symbol,station_name_y,station_latitude,station_longitude,site_no,site_name,object_type_shortname,catchment_name,sh_district,station_no_parameter
0,LAU03,Auburn Lake - East,ChlA,ChlA,2019-09-16T13:00:00.000Z,45.100,µg/l,Auburn Lake - East,44.8639,-93.67960,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;LK_STATION,---,---,LAU03-ChlA
1,LAU03,Auburn Lake - East,ChlA,ChlA,2019-07-22T13:15:00.000Z,48.400,µg/l,Auburn Lake - East,44.8639,-93.67960,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;LK_STATION,---,---,LAU03-ChlA
2,LAU03,Auburn Lake - East,ChlA,ChlA,2019-06-24T10:30:00.000Z,26.300,µg/l,Auburn Lake - East,44.8639,-93.67960,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;LK_STATION,---,---,LAU03-ChlA
3,LAU03,Auburn Lake - East,ChlA,ChlA,2019-08-08T08:20:00.000Z,76.800,µg/l,Auburn Lake - East,44.8639,-93.67960,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;LK_STATION,---,---,LAU03-ChlA
4,LAU03,Auburn Lake - East,ChlA,ChlA,2019-07-08T11:15:00.000Z,43.500,µg/l,Auburn Lake - East,44.8639,-93.67960,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;LK_STATION,---,---,LAU03-ChlA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6075,LWS01,Wassermann Lake,TP,TP,2019-07-25T12:45:00.000Z,0.691,mg/l,Wassermann Lake,44.8410,-93.67371,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;A_STATION;LK_STATION,---,---,LWS01-TP
6076,LWS01,Wassermann Lake,TP,TP,2019-08-21T10:50:00.000Z,1.030,mg/l,Wassermann Lake,44.8410,-93.67371,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;A_STATION;LK_STATION,---,---,LWS01-TP
6077,LWS01,Wassermann Lake,TP,TP,2019-09-16T10:20:00.000Z,0.093,mg/l,Wassermann Lake,44.8410,-93.67371,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;A_STATION;LK_STATION,---,---,LWS01-TP
6078,LWS01,Wassermann Lake,TP,TP,2019-06-12T10:00:00.000Z,0.300,mg/l,Wassermann Lake,44.8410,-93.67371,SMC,Six Mile Creek Subwatershed,GN_STATION;WQ_STATION;A_STATION;LK_STATION,---,---,LWS01-TP


In [100]:
#calculate annual averages
result = WQ_subset.groupby('station_no_parameter', as_index=False).mean()
result


,station_no_parameter,value,station_latitude,station_longitude
0,LAU03-ChlA,41.750000,44.863900,-93.679600
1,LAU03-SD,1.068750,44.863900,-93.679600
2,LAU03-TP,0.195438,44.863900,-93.679600
3,LCH01-ChlA,1.111429,44.899200,-93.537800
4,LCH01-SD,8.471429,44.899200,-93.537800
...,...,...,...,...
64,LVI01-SD,1.285714,44.883296,-93.635697
65,LVI01-TP,0.427000,44.883296,-93.635697
66,LWS01-ChlA,52.175000,44.841000,-93.673710
67,LWS01-SD,0.890000,44.841000,-93.673710


In [101]:
#iterate through keys to find original lake names and lab analysis unit values
units = []
lake_name = []

for i in result.station_no_parameter:
    units.append(WQ_subset['unit_symbol'][WQ_subset['station_no_parameter'] == i].iloc[0])
    lake_name.append(WQ_subset['station_name_y'][WQ_subset['station_no_parameter'] == i].iloc[0])
result.insert(2, 'unit_symbol', units)
result.insert(1, 'station_name_y', lake_name)

In [102]:
split_data = result.station_no_parameter.str.split('-', expand=True).rename({0: 'Site', 1: 'Parameter'}, axis=1)
split_data

,Site,Parameter
0,LAU03,ChlA
1,LAU03,SD
2,LAU03,TP
3,LCH01,ChlA
4,LCH01,SD
...,...,...
64,LVI01,SD
65,LVI01,TP
66,LWS01,ChlA
67,LWS01,SD


In [103]:
#combine the split columns and result dataframe
wq_combined = pd.concat([split_data, result], axis=1)
wq_combined

,Site,Parameter,station_no_parameter,station_name_y,value,unit_symbol,station_latitude,station_longitude
0,LAU03,ChlA,LAU03-ChlA,Auburn Lake - East,41.750000,µg/l,44.863900,-93.679600
1,LAU03,SD,LAU03-SD,Auburn Lake - East,1.068750,m,44.863900,-93.679600
2,LAU03,TP,LAU03-TP,Auburn Lake - East,0.195438,mg/l,44.863900,-93.679600
3,LCH01,ChlA,LCH01-ChlA,Christmas Lake,1.111429,µg/l,44.899200,-93.537800
4,LCH01,SD,LCH01-SD,Christmas Lake,8.471429,m,44.899200,-93.537800
...,...,...,...,...,...,...,...,...
64,LVI01,SD,LVI01-SD,Lake Virginia,1.285714,m,44.883296,-93.635697
65,LVI01,TP,LVI01-TP,Lake Virginia,0.427000,mg/l,44.883296,-93.635697
66,LWS01,ChlA,LWS01-ChlA,Wassermann Lake,52.175000,µg/l,44.841000,-93.673710
67,LWS01,SD,LWS01-SD,Wassermann Lake,0.890000,m,44.841000,-93.673710


In [104]:
#Convert coordinates  into point data for GIS processing
wq_combined['coordinates'] = wq_combined[['station_longitude', 'station_latitude']].values.tolist()
wq_combined['coordinates'] = wq_combined['coordinates'].apply(Point)
Lake_WQ_Data = geopandas.GeoDataFrame(wq_combined, geometry='coordinates')

In [105]:
Lake_WQ_Data

,Site,Parameter,station_no_parameter,station_name_y,value,unit_symbol,station_latitude,station_longitude,coordinates
0,LAU03,ChlA,LAU03-ChlA,Auburn Lake - East,41.750000,µg/l,44.863900,-93.679600,POINT (-93.67959999999999 44.8639)
1,LAU03,SD,LAU03-SD,Auburn Lake - East,1.068750,m,44.863900,-93.679600,POINT (-93.67959999999999 44.8639)
2,LAU03,TP,LAU03-TP,Auburn Lake - East,0.195438,mg/l,44.863900,-93.679600,POINT (-93.67959999999997 44.86390000000002)
3,LCH01,ChlA,LCH01-ChlA,Christmas Lake,1.111429,µg/l,44.899200,-93.537800,POINT (-93.53779999999999 44.8992)
4,LCH01,SD,LCH01-SD,Christmas Lake,8.471429,m,44.899200,-93.537800,POINT (-93.53779999999999 44.8992)
...,...,...,...,...,...,...,...,...,...
64,LVI01,SD,LVI01-SD,Lake Virginia,1.285714,m,44.883296,-93.635697,POINT (-93.63569735999999 44.88329583)
65,LVI01,TP,LVI01-TP,Lake Virginia,0.427000,mg/l,44.883296,-93.635697,POINT (-93.63569735999999 44.88329582999999)
66,LWS01,ChlA,LWS01-ChlA,Wassermann Lake,52.175000,µg/l,44.841000,-93.673710,POINT (-93.67371000000001 44.841)
67,LWS01,SD,LWS01-SD,Wassermann Lake,0.890000,m,44.841000,-93.673710,POINT (-93.67371000000001 44.841)
